In [96]:
import numpy as np
def showarray(a, fmt='png'):
    a = np.uint8(a)
    f = StringIO()
    PIL.Image.fromarray(a).save(f, fmt)
    IPython.display.display(IPython.display.Image(data=f.getvalue()))
class GF():
    def __init__(self, f):
        Max = np.max(f)
        f2 = np.zeros(Max+1, np.uint8)
        f2[f] = 1
        f = np.flip(f2)
        self.len_of_symbol = Max
        self.q = 2**self.len_of_symbol
        self.table = np.array(self.make_code_table(f))
        
    def sub_matrix(self, A, i, j):
        A1 = np.delete(A, i, 0)
        A2 = np.delete(A1, j, 1)
        return A2

    def det(self, A):
        size = A.shape
        if size[0]==2 and size[1]==2:
            d = self.add(self.multi(A[0, 0],A[1, 1]), self.multi(A[1, 0],A[0, 1]))
            return d
        else:
            d = 0
            for i in range(size[0]):
                d=self.add(self.multi(A[0, i],self.det(self.sub_matrix(A, 0, i))), d)
            return d
        
    def make_code_table(self, f):
        m = len(f)-1
        N = 2**m
        table = ["0"*m]
        x =""
        for i in range(1,m+1):
            if f[i]==0:
                x+="0"
            else:
                x+="1"
        for i in range(0, m):
            c="0"*m
            c = c[0:i] + "1"+c[i+1:]
            c = c[::-1]
            table.append(c)
        for i in range(m+1, N):
            a = table[-1]
            a1 = a[1:]+"0"
            b = "0"*m
            if a[0]=="1":
                b = x
            c = self.add_string_code(a1, b)
            table.append(c)
        return table

    def add_string_code(self, a, b):
        c =""
        for i in range(self.len_of_symbol):
            if a[i]!= b[i] :
                c+="1"
            else:
                c+="0"
        return c
    def add(self, a, b):
        if  np.isscalar(a) and  np.isscalar(b):
            a= int(a)
            b = int(b)
            a1 = self.table[a]
            b1 = self.table[b]
            c1 = self.add_string_code(a1, b1)
            c, = np.where(self.table == c1)
            return c[0]
        elif not np.isscalar(a):
            if np.isscalar(b):
                raise ValueError("a, b must have same shape ( a:",a.shape,", b is scalar)" )
            else:
                a_size = a.shape
                b_size = b.shape
                if  len(a_size) ==2 and  len(a_size)==2:
                    if a_size[0] != b_size[0] or a_size[1] != b_size[1] :
                        raise ValueError("a, b must have same shape ( a:"+ str(a.shape) +",  b:"+str(b.shape)+ ")" )
                    else:
                        c = np.zeros_like(a, np.uint8)
                        for i in range(a_size[0]):
                            for j in range(a_size[1]):
                                c[i, j] = self.add(a[i, j], b[i, j])
                        return c
                elif len(a_size)==1 and len(b_size)==1 and a_size ==b_size :
                    c = np.zeros_like(a, np.uint8)
                    for i in range(a_size[0]):
                        c[i] = self.add(a[i], b[i])
                    return c
                else:
                    raise ValueError("a, b must have same shape ( a:"+ str(a.shape) +",  b:"+str(b.shape)+ ")" )         
    def multi(self, a, b):
        if  np.isscalar(a) and  np.isscalar(b):
                if a ==0 or b == 0:
                    return 0
                else:
                    c = (a+b-2) % (self.q-1)
                    return c+1
        elif not np.isscalar(a):
            a_size = a.shape
            if np.isscalar(b):
                c = [self.multi(i, b) for i in a]
                return np.array(c)
            b_size = b.shape
            if a_size[1] != b_size[0] :
                raise ValueError("Can not multiply 2 matrix of dimension: ", a.shape, b.shape)
            else:
                c = np.zeros((a_size[0], b_size[1]), np.uint8)
                for i in range(a_size[0]):
                    for j in range(b_size[1]):
                        for k in range(a_size[1]):
                            c[i, j] =self.add(c[i, j], self.multi(a[i, k], b[k, j])) 
                return c
    def make_sub_matrix(self, x):
        N =  N = self.q -1
        sub = np.zeros((N, N), np.uint8)
        index = x-1
        for i in range(N):
            sub[i, index] = index+1
            index +=1
            if index ==N:
                index =0
        return sub
    def reserve_e(self, a):
            a2 = self.q+1-a
            if a2 == self.q:
                a2 = 1
            if a2 == self.q+1:
                a2 = 0
            return a2
    def Reserve(self, X, show =False):
        N = X.shape[0]
        def make_I(n):
                In = np.zeros((n, n), np.uint8)
                for i in range(n):
                    In[i, i] =1
                return In
        X1 = make_I(N)
        for i in range(N):
            if show:
                print(i)
            C = make_I(N)
            for j in range(N):
                C[j, i] = self.multi(X[j, i], self.reserve_e(X[i,i]))
            C[i,i] = self.reserve_e(X[i,i])
            X = self.multi(C,X)
            X1 = self.multi(C,X1)
        return X1
    
import numpy as np
from scipy.special import erfc

def S2B(symbol, q):
    b = bin(symbol)[2:].zfill(q)
    B = np.array(list(b)).astype(np.float32)
    return B

def probB(x, N0):
    d1 = abs(x)
    d2 = abs(1-x)
    p0 = np.e**(-d1/N0)/(np.e**(-d1/N0)+np.e**(-d2/N0))
    p1 = 1-p0
    return [p0, p1]

def probS(S, N0, q, M):
    B = np.zeros((q, 2))
    for i in range(q):
        B[i] = probB(S[i], N0)
    P = np.zeros(M)
    for i in range(M):
        Bi = S2B(i, q)
        p = 1
        for j in range(q):
            p *=B[j, int(Bi[j])]
        P[i] = p
    m = np.max(P)
    P = np.log10(m/P)
    return P

def BPSK(message, M, SNRdB):
    Eb = 0.5
    N0 = Eb/10**(SNRdB/10)
    q = int(np.log2(M))
    Ns = len(message)
    Nb = Ns*q
    noise = np.sqrt(N0/2)*np.random.randn(Nb)
    noise = noise.reshape((Ns, q))
    X = np.zeros_like(noise)
    for i in range(Ns):
        X[i]= S2B(message[i], q)
    Y = X+noise
    output = np.zeros((Ns, M))
    for i in range(Ns):
        output[i] = probS(Y[i], N0, q, M)
    output[output>6]=6
    return output

def HardDicision(Message):
    return np.argmin(Message, axis=1)

def score(X, Y):
    Z = np.zeros_like(X)
    Z[X!=Y]=1
    return np.sum(Z)

In [97]:
def predict(Q):
    message = np.argmin(Q, axis=0)
    X = 0
    for i in range(dc):
        X = gl4.add(X, message[i])
    print("Syndrome", X)
    return message, X

def convert_vhdl(A, elem32=False):
    shape = A.shape
    s = "("
    if len(shape)==1:
        for i in A:
            si = int(i)
            s+=str(si)+", "
        s=s[:-2]+")"
        return s
    if len(shape)==2:
        for row in A:
            print(convert_vhdl(row)+",")
gl4 = GF([5,  2, 0])
print(gl4.table)


['00000' '00001' '00010' '00100' '01000' '10000' '00101' '01010' '10100'
 '01101' '11010' '10001' '00111' '01110' '11100' '11101' '11111' '11011'
 '10011' '00011' '00110' '01100' '11000' '10101' '01111' '11110' '11001'
 '10111' '01011' '10110' '01001' '10010']


In [149]:
dc = 27
q = 32
A = np.random.randint(1, 100, (q, dc))*10
A = A.astype(np.float32)
for i in range(dc):
    s = np.sum(A[:, i])
    A[:,i]=A[:,i]*100/s
    i_min = np.argmax(A[:, i])
    A[i,i], A[i_min,i] = A[i_min,i], A[i,i] 
index= np.random.permutation(q)
A[:, 0] = A[index, 0]

Q = np.zeros_like(A, np.float32)
for i in range(dc):
    m = np.max(A[:, i])
    for j in range(q):
        Q[j,i] = np.log10(m/A[j, i])
Q = (Q*1000).astype(np.int32)
zn = predict(Q)
print(zn)
convert_vhdl(Q)

Syndrome 11
(array([ 4,  1,  2,  3,  4,  5,  6,  7,  3,  9, 10, 11,  2, 13, 14, 15, 16,
       15, 18, 19, 15, 21, 22, 23, 24, 25, 26], dtype=int64), 11)
(196, 238, 296, 155, 223, 916, 1633, 31, 393, 352, 378, 454, 63, 257, 203, 253, 246, 32, 713, 439, 849, 247, 541, 223, 806, 986, 726),
(13, 0, 1180, 626, 430, 881, 291, 1393, 360, 362, 328, 441, 287, 510, 76, 74, 47, 411, 296, 27, 548, 1694, 826, 644, 169, 1287, 672),
(126, 986, 0, 99, 555, 1296, 788, 31, 769, 404, 104, 160, 0, 1371, 189, 151, 279, 400, 205, 653, 4, 1518, 650, 141, 836, 287, 269),
(17, 343, 251, 0, 129, 995, 390, 41, 0, 51, 367, 893, 245, 797, 255, 397, 1509, 56, 491, 263, 533, 765, 265, 986, 438, 406, 505),
(0, 872, 34, 521, 0, 633, 390, 1393, 166, 393, 133, 429, 151, 1018, 46, 74, 1986, 1213, 335, 56, 393, 564, 173, 395, 726, 1685, 162),
(71, 246, 132, 129, 588, 0, 20, 176, 496, 849, 499, 717, 206, 672, 232, 386, 395, 912, 588, 1995, 150, 362, 257, 123, 903, 262, 33),
(314, 13, 578, 39, 57, 615, 0, 1694, 454, 791, 1

In [150]:
dc = 27
q = 32
SNRdB = 1
A = np.arange(27)
print(A)
Q0 = BPSK(A, q, SNRdB)
Q0 = Q0.transpose()

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26]


In [163]:
Q = (Q0*1000).astype(np.uint16)
print(np.max(Q))
zn, C = predict(Q)
print(zn)

5467
Syndrome 7
[ 4  0  2  7  6  5  7  7  9 11 30  7 12 13  6 15 24 17 31 19 21 21 30 23
 24 25 26]


In [175]:
def debug(name, value, debug=False):
    if debug:
        print(name+"#"*30)
        print(value)
    
def CN(Q, dg=False):
    A = 1
    zn = np.argmin(Q, axis=0)
    debug("Zn", zn, dg)
    dQ = np.zeros_like(Q)
    for j in range(dc):
        for a in range(q):
            nj = gl4.add(a, zn[j])
            dQ[nj, j] = Q[a, j]
            
    debug("dQ", dQ, dg)
    B = 0
    for i in range(dc):
        B = gl4.add(B, zn[i])
    debug("B", B, dg)
    def sai(A):
        col = np.argmin(A)
        sortA = np.sort(A)
        return sortA[0], sortA[1], col

    m = np.zeros((q-1, 3))
    for i in range(1, q):
        m[i-1]= sai(dQ[i])
    debug("m", m, dg)
    debug("m0", m[:, 0], dg)
    print(convert_vhdl(m[:, 0]))        
    print("m1"+"#"*30)
    print(convert_vhdl(m[:, 1], True))
    conf = np.ones((q, q))*1000000
    conf[0, 0] = 0
    conf[1:, 0] = m[:, 0]
    for i in range(1,q-1):
        for j in range(i+1, q):
            if m[i-1, 2]!=m[j-1,2]:
                conf[gl4.add(i,j), i] = max(m[i-1, 0],m[j-1,0])
    debug("conf", conf, dg)
    extra_columm = np.zeros((q, 5))
    extra_columm[:, 0] = np.min(conf, axis=1)
    extra_columm[1:, 1] = m[:, 0]
    extra_columm[1:, 2] = m[:, 1]
    argmin=  np.argmin(conf, axis=1)
    for i in range(1, q):
        if argmin[i]==0:
            extra_columm[i, 3] = extra_columm[i, 4] = m[i-1, 2]
        else:
            d1 = m[argmin[i]-1, 2]
            d2 = m[gl4.add(argmin[i], i)-1, 2]
            extra_columm[i, 3]=d1
            extra_columm[i, 4] =d2
    debug("extra_columm", extra_columm, dg)
    dR = np.zeros_like(dQ)
    
    def dRij(extra_colummi, j):
        dQi, m1, m2, d1, d2 = extra_colummi
        if d1==j or d2==j:
            return m2 if d1==d2 else m1
        else:
            return dQi
        
    for i in range(q):
        for j in range(dc):
            dR[i, j] = dRij(extra_columm[i], j)
    debug("dR", dR, dg)
    R = np.zeros_like(Q)
    for i in range(q):
        for j in range(dc):
            index = gl4.add(i, B)
            index = gl4.add(index, zn[j])
            R[index, j]= A*dR[i,j]
            
    debug("R", R,dg) 
    Q = Q+R
    debug("Q", Q, dg)
    return Q

Q = CN(Q, True)
C,X= predict(Q)
print(C)

Zn##############################
[20 24 17  3 14 25  3  3 10  8 12  3 30 27  6  3  1 23 26 10 23 23 12 10
 29  8 27]
dQ##############################
[[  604  1669  2738   144   551  1863   604   201   847   833   432    10
   1343  2390   897  1644   883  1697   833   534   551  1992  1130  2929
    223   244   597]
 [18476 20955 18468 19178 19484 20406 18307 18470 18409 21232 18660 18470
  18210 19207 18764 19301 17782 19708 18101 18109 20998 21283 18372 18736
  18417 19520 19792]
 [40110 39836 40257 40469 42988 40480 40645 40691 42830 40398 39697 41309
  38907 41078 39519 41835 40864 39620 39720 40061 41344 40638 39663 39164
  39116 39900 41295]
 [ 4338  6834  5552  7290  4867  6630  6734  6494  5428 10420  4843  6166
   5423  8485  5928  7137  7216  5540  5176  5232  8255  7420  5860  5702
   6791  8077  4710]
 [40569 41912 39149 39482 40863 39497 39775 39684 42118 40039 39763 40275
  41022 42326 41236 40768 40995 41829 39848 39463 40917 40716 39833 39227
  38566 39918 39832]
 [414

In [ ]:
def CN1(Q):
    A = 1
    zn = np.argmin(Q, axis=0)
    print("Zn"+"#"*30)
    print(zn)
    dQ = np.zeros_like(Q)
    for j in range(dc):
        for a in range(q):
            nj = gl4.add(a, zn[j])
            dQ[nj, j] = Q[a, j]
    print("dQ"+"#"*30)
    print(dQ)
    print("B"+"#"*30)
    B = 0
    for i in range(dc):
        B = gl4.add(B, zn[i])
    print(B)
    def sai(A):
        col = np.argmin(A)
        sortA = np.sort(A)
        return sortA[0], col, sortA[1]

    m = np.zeros((q-1, 3))
    for i in range(1, q):
        m[i-1]= sai(dQ[i])
    print("m"+"#"*30)
    print(m)
    print("m0"+"#"*30)
    print(convert_vhdl(m[:, 0]))
    print("m1"+"#"*30)
    print(convert_vhdl(m[:, 1], True))
    conf = np.ones((q, q))*1000
    conf[0, 0] = 0
    conf[1:, 0] = m[:, 0]
    for i in range(1,q-1):
        for j in range(i+1, q):
            if m[i-1, 1]!=m[j-1,1]:
                conf[gl4.add(i,j), i] = max(m[i-1, 0],m[j-1,0])
    print("conf"+"#"*30)
    print(conf)
    extra_columm = np.min(conf, axis=1)
    print("extra_columm"+"#"*30)
    print(extra_columm)
    dR = np.zeros_like(dQ)
    for i in range(1, q):
        for j in range(dc):
            if dQ[i,j]!=m[i-1, 0] and dQ[i,j]!=m[i-1, 2]:
                dR[i,j]=extra_columm[i]
            elif dQ[i,j]==m[i-1, 0]:
                 dR[i,j] = m[i-1, 2]
            else:
                dR[i,j] = m[i-1, 0]
    print("dR"+"#"*30)
    print(dR)
    R = np.zeros_like(Q)
    for i in range(q):
        for j in range(dc):
            index = gl4.add(i, B)
            index = gl4.add(index, zn[j])
            R[index, j]= A*dR[i,j]
    print("R"+"#"*30)
    print(R)    
    Q = Q+R
    print("Q"+"#"*30)
    print(Q)
    return Q
def CN2(Q):
    A = 1
    zn = np.argmin(Q, axis=0)
    print("Zn"+"#"*30)
    print(zn)
    dQ = np.zeros_like(Q)
    for j in range(dc):
        for a in range(q):
            nj = gl4.add(a, zn[j])
            dQ[nj, j] = Q[a, j]
    print("dQ"+"#"*30)
    print(dQ)
    print("B"+"#"*30)
    B = 0
    for i in range(dc):
        B = gl4.add(B, zn[i])
    print(B)
    def sai(A):
        col = np.argmin(A)
        sortA = np.sort(A)
        return sortA[0], col, sortA[1]

    m = np.zeros((q-1, 3))
    for i in range(1, q):
        m[i-1]= sai(dQ[i])
    print("m"+"#"*30)
    print(m)
    print("m0"+"#"*30)
    print(convert_vhdl(m[:, 0]))
    print("m1"+"#"*30)
    print(convert_vhdl(m[:, 1], True))
    conf = np.ones((q, q))*1000
    conf[0, 0] = 0
    conf[1:, 0] = m[:, 0]
    for i in range(1,q-1):
        for j in range(i+1, q):
            if m[i-1, 1]!=m[j-1,1]:
                conf[gl4.add(i,j), i] = max(m[i-1, 0],m[j-1,0])
    print("conf"+"#"*30)
    print(conf)
    extra_columm = np.min(conf, axis=1)
    print("extra_columm"+"#"*30)
    print(extra_columm)
    dR = np.zeros_like(dQ)
    argmin=  np.argmin(conf, axis=1)
    for i in range(1, q):
        index = argmin[i]
        for j in range(dc):
            dR[i, j]=extra_columm[i]
        if index==0:
            dR[i, int(m[i-1, 1])]=m[i-1, 2]
        else:
            dR[i, int(m[index-1, 1])]=m[i-1, 0]
            
            dR[i, int(m[gl4.add(index, i)-1, 1])]=m[i-1, 0]
    print("dR"+"#"*30)
    print(dR)
    R = np.zeros_like(Q)
    for i in range(q):
        for j in range(dc):
            index = gl4.add(i, B)
            index = gl4.add(index, zn[j])
            R[index, j]= A*dR[i,j]
    print("R"+"#"*30)
    print(R)    
    Q = Q+R
    print("Q"+"#"*30)
    print(Q)
    return Q
Q2 = CN1(Q)
C1,X1= predict(Q2)
Q2 = CN2(Q)
C2,X2= predict(Q2)
print(C1)
print(C2)
print(X1, X2)